In [1]:
import pygame
import sys
import random
import math

from pygame.locals import *

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# # setup game
# mainClock = pygame.time.Clock()


# create window
WINDOWWIDTH = 800
WINDOWHEIGHT = 800


# resources
BRICK = (255, 0, 0)
ORE = (190, 190, 190)
WHEAT = (255, 215, 0)
SHEEP = (124, 252, 0)
WOOD = (0, 100, 0)
DESSERT = (139, 69, 19)

resources = {BRICK: 'Brick', ORE: 'Ore', WHEAT : 'Wheat',
            SHEEP : 'Sheep', WOOD:'Wood', DESSERT:'Dessert'}

# players
P1 = (0, 0, 255)
P2 = (255, 20, 147)
P3 = (255, 165, 0)
P4 = (70, 130, 180)

# hex locations
hexTiles = [(0,0), (0,1), (0,-1), (-1,0.5), (-1,-0.5), (1,0.5), (1,-0.5), (2,0), (-2,0), (0,2),
                (0,-2), (1,1.5), (1,-1.5), (-1,1.5), (-1, -1.5), (2,1), (2,-1), (-2,1), (-2,-1)]

hexProbabilities = [5, 2, 6, 3, 8, 10, 9, 12, 11, 4, 8, 10, 9, 4, 5, 6, 3, 11]


probabilities_values = {
    2 : 1,
    3 : 2,
    4 : 3,
    5 : 4,
    6 : 5,
    7 : 0,
    8 : 5,
    9 : 4,
    10 : 3,
    11 : 2,
    12 : 1,
}

resource_values = {
    'Brick' : 1,
    'Ore' : 3,
    'Wheat' : 3,
    'Sheep' : 2,
    'Wood' : 2,
    'Dessert' : 0
}

position_names = {
 '315395': 'AA',
 '355325': 'AB',
 '435325': 'AC',
 '475395': 'AD',
 '435455': 'AE',
 '355455': 'AF',
 '315525': 'AG',
 '475525': 'AH',
 '435595': 'AI',
 '355595': 'AJ',
 '315255': 'AK',
 '355185': 'AL',
 '435185': 'AM',
 '475255': 'AN',
 '195455': 'AO',
 '235395': 'AP',
 '235525': 'AQ',
 '195325': 'AR',
 '235255': 'AS',
 '555395': 'AT',
 '595455': 'AU',
 '555525': 'AV',
 '555255': 'AW',
 '595325': 'AX',
 '675325': 'AY',
 '715395': 'AZ',
 '675455': 'BA',
 '75395':  'BB',
 '115325': 'BC',
 '115455': 'BD',
 '315665': 'BE',
 '475665': 'BF',
 '435735': 'BG',
 '355735': 'BH',
 '315115': 'BI',
 '35545':  'BJ',
 '43545':  'BK',
 '475115': 'BL',
 '595595': 'BM',
 '555665': 'BN',
 '555115': 'BO',
 '595185': 'BP',
 '195595': 'BQ',
 '235665': 'BR',
 '195185': 'BS',
 '235115': 'BT',
 '715525': 'BU',
 '675595': 'BV',
 '675185': 'BW',
 '715255': 'BX',
 '75525':  'BY',
 '115595': 'BZ',
 '75255':  'CA',
 '115185': 'CB'}



In [12]:
# Board Class
class Board:
    hexes = []
    settlements = []
    instructions = ""
    
    def __init__(self, size, screen, zipHex, settlementLoc):
        self.screen = screen
        self.cX = self.screen.get_rect().centerx
        self.cY = self.screen.get_rect().centery
        
        self.hexHeight = 0.866 * size
        
        # the hexs
        self.hexes=[]
        for tile in zip(hexTiles, zipHex):
            self.hexes.append(Hex(tile[0], size, tile[1][0], tile[1][1], self, settlementLoc))
        
        # set up the settlement locations
        self.settlements = settlementLoc 

            
    def draw(self):
        # draw the HEXes
        for h in self.hexes:
            h.draw()

        # draw the settlement locations
        for item in self.settlements:
            item.draw()

        # message to user
        basicFont = pygame.font.SysFont(None, 24)
        text = basicFont.render("HEllo", True, (255,255,255), (0,0,0))
        textRect = text.get_rect()
        textRect.left = 5
        textRect.top = 560
        self.screen.blit(text, textRect)
        
        
        
""" Create a settlement class"""
class Settlement:
    def __init__(self, location, tileOwner):
        self.location = location
        self.locationx = location[0]
        self.locationy = location[1]
        self.id = position_names[str(location[0])+str(location[1])]
        self.onTiles = [tileOwner]
        self.neighbours = [] # do I need?
        self.city = False
        self.owner = -1
        self.check_coords = [
            str(self.locationx + 80)+ str(self.locationy),
            str(self.locationx - 80)+ str(self.locationy),
            str(self.locationx + 40)+ str(self.locationy + 70),
            str(self.locationx + 40)+ str(self.locationy - 70),
            str(self.locationx - 40)+ str(self.locationy + 70),
            str(self.locationx - 40)+ str(self.locationy - 70),
            str(self.locationx + 40)+ str(self.locationy + 60),
            str(self.locationx + 40)+ str(self.locationy - 60),
            str(self.locationx - 40)+ str(self.locationy + 60),
            str(self.locationx - 40)+ str(self.locationy - 60),

        ]
        
        self.omit_list = []
        for i in self.check_coords:
            try:
                self.omit_list.append(position_names[i])
            except:
                pass


        
    def draw(self):
#         pygame.draw.rect(screen, (0,0,0), (self.locationx, self.locationy, 20, 20),2)
        

        font = pygame.font.Font(None, 20)
        text = font.render(str(self.id), 1, (255,255,255), (0,0,0))
        textpos = text.get_rect()
        textpos.left = self.locationx + 3
        textpos.top = self.locationy + 3
        screen.blit(text, textpos)


        
# Hex Class
class Hex:

    def __init__(self, centre, size, resource, probability, board, settlementLoc):
        self.centre = centre
        self.hexX = board.cX +(1.5*centre[0]*size)
        self.hexY = board.cY+(2*centre[1]*board.hexHeight)
        self.hexSize = size
        self.resource = resource
        self.value = probability	# chance of getting this resource
        self.hasRobber = False
        
        # calculate each 6 points so we only do it once
        self.points = []
        self.points.append((self.hexX - self.hexSize, self.hexY))
        self.points.append((self.hexX - (self.hexSize/2), self.hexY - (0.866*self.hexSize)))
        self.points.append((self.hexX + (self.hexSize/2), self.hexY - (0.866*self.hexSize)))
        self.points.append((self.hexX + self.hexSize, self.hexY))
        self.points.append((self.hexX + (self.hexSize/2), self.hexY + (0.866*self.hexSize)))
        self.points.append((self.hexX - (self.hexSize/2), self.hexY + (0.866*self.hexSize)))
        
        # add the points of all settlements to the settlement lists
#         global settlementLoc
        newSets = [] # store all the locations of the new settlements from this til
        newSets += [((10*math.floor(X / 10))-5, (10*math.floor(Y / 10))-5) for (X,Y) in self.points]
        
        for ns in newSets: # these are the ones for the current hex
            for s in settlementLoc: # these are the ones that have been established
                if s.location == ns:  # so if currently is an established settlement
                    newSets.remove(ns)  # remove it from the set
                    s.onTiles.append(self)  # add to that settlement's tile owneres
                    break
                    
        for ns in newSets:	# these are the remainders
            settlementLoc.append(Settlement(ns, self))
        
    def draw(self): 
        # draw the hex
        pygame.draw.polygon(screen, self.resource, self.points, 0)
        
        # draw the text
        if self.value > 0:
            font = pygame.font.Font(None, 36)
            text = font.render(str(self.value), 1, (0,0,0))
            textpos = text.get_rect()
            textpos.left = self.hexX - 5
            textpos.top = self.hexY - 10
            screen.blit(text, textpos)

class Player:
    def __init__(self, turn, bot):
        self.turn = turn
        self.value = 0
        self.bot = True
        
    def action(self, choice, choices, settlers, pos_values):
        
        # if it's a bot, then selects random option from possible position
        if self.bot:
            choice = random.choice(choices)
        else:
            choice = choice
            
            
        # removes choice from possible choices
        choices.remove(choice)
        
        # removes adjacent choices from possible choices
        for sett in settlers.settlements:
            if sett.id == choice:
                try:
                    [choices.remove(i) for i in sett.omit_list]
                except:
                    pass
                break
                
        # updates player value
        self.value += pos_values[choice]
        print('Player {0} has {1} points'.format(self.turn, self.value))
        
        return self.value
    
    
class CatanEnv:
    def __init__(self):
        pass
#         self.player1 = Player(1, bot=True)
#         self.player2 = Player(2, bot=True)
#         self.player3 = Player(3, bot=True)
#         self.player4 = Player(4, bot=True)
        
#         self.settlers = Board(80)
        
        
    
    def setup(self):
        
        settlementLoc = []
        screen = pygame.display.set_mode((WINDOWWIDTH,WINDOWHEIGHT), 0, 32)

        screen.fill((0,194,249))

        pygame.display.set_caption('Catan')
        
        random.shuffle(hexProbabilities)

        hexResources = [BRICK, BRICK, BRICK, ORE, ORE, ORE, WHEAT, WHEAT, WHEAT, WHEAT, SHEEP, 
                        SHEEP, SHEEP, SHEEP, WOOD, WOOD, WOOD, WOOD]
        
        zipHex = []

        zipHex = list(zip(hexResources, hexProbabilities))

        zipHex.append((DESSERT, 7))


        random.shuffle(zipHex)


        
        self.player1 = Player(1, bot=True)
        self.player2 = Player(2, bot=True)
        self.player3 = Player(3, bot=True)
        self.player4 = Player(4, bot=True)

        pos_values = {}

        turn = 1
        done = False
        settlers = Board(80, screen, zipHex, settlementLoc)

        
        for position in settlers.settlements:
            for tile in position.onTiles:
                try:
                    pos_values[position.id] += probabilities_values[tile.value] * resource_values[str(resources[tile.resource])] 
#                     print(tile.value)
                except:
                    pos_values[position.id] = probabilities_values[tile.value] * resource_values[str(resources[tile.resource])] 
                    
        choices = [k for k, _ in pos_values.items()]
        
        return turn, done, choices, settlers, pos_values
        
        
    def step(self, turn, choice, choices, settlers, pos_values, done):
        if turn == 1:
            self.player1.action(choice, choices, settlers, pos_values)
            turn += 1
        
        elif turn == 2:
            self.player2.action(choice, choices, settlers, pos_values)
            turn += 1
            
        elif turn == 3:
            self.player3.action(choice, choices, settlers, pos_values)
            turn += 1
            
        elif turn == 4:
            self.player4.action(choice, choices, settlers, pos_values)
            turn += 1

        elif turn == 5:
            self.player4.action(choice, choices, settlers, pos_values)
            turn += 1

        elif turn == 6:
            self.player3.action(choice, choices, settlers, pos_values)
            turn += 1

        elif turn == 7:
            self.player2.action(choice, choices, settlers, pos_values)
            turn += 1

        elif turn == 8:
            self.player1.action(choice, choices, settlers, pos_values)
            turn += 1
        
        else:
            #ends the game
            done = True
            pass
        
        player_values = [self.player1.value, self.player2.value, self.player3.value, self.player4.value]
        
        return turn, done, player_values



In [13]:
test = CatanEnv()
choice = ''


In [45]:
t, d, choices, settlers, pos_values = test.setup()

# print(t, d, choices, pos_values)
while not d:
    t, d, pv = test.step(t,choice, choices, settlers, pos_values, d)
    
print(pv)

Player 1 has 15 points
Player 2 has 12 points
Player 3 has 6 points
Player 4 has 10 points
Player 4 has 19 points
Player 3 has 6 points
Player 2 has 34 points
Player 1 has 38 points
[38, 34, 6, 19]


In [36]:
""" Set up the information for the settlements """
settlers = Board(80)


pos_values = {}

for position in settlers.settlements:
    for tile in position.onTiles:
        try:
            pos_values[position.id] += probabilities_values[tile.value] * resource_values[str(resources[tile.resource])] 
        except:
            pos_values[position.id] = probabilities_values[tile.value] * resource_values[str(resources[tile.resource])] 
        
choices = [k for k, _ in pos_values.items()]

# {k: v for k, v in sorted(pos_values.items(), key=lambda item: item[1])}


TypeError: __init__() missing 3 required positional arguments: 'screen', 'zipHex', and 'settlementLoc'

In [ ]:

# run the game loop
pygame.init()
p1 = Player(1, bot=False)
p2 = Player(2, bot=True)
p3 = Player(3, bot=True)
p4 = Player(4, bot=True)

while not done:
    # check for the QUIT event
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()
            
            
#         elif event.type == MOUSEBUTTONDOWN:
        else:
            # player 1
            choice = random.choice(choices)
            print("choice = {}".format(choice))
            choices.remove(choice)
            for sett in settlers.settlements:
                if sett.id == choice:
                    # A transparent surface with per-pixel alpha.
                    circle = pygame.Surface((60, 60), pygame.SRCALPHA)
                    # Draw a circle onto the `circle` surface.
                    pygame.draw.circle(circle, P1, [20, 20], 30)                    
                    screen.blit(circle, [sett.locationx, sett.locationy])
                    try:
                        [choices.remove(i) for i in sett.omit_list]
                    except:
                        pass
                    break
            
            p1.action(choice)
            
            # player 2
            choice = random.choice(choices)
            print("choice = {}".format(choice))
            choices.remove(choice)
            for sett in settlers.settlements:
                if sett.id == choice:

                    try:
                        [choices.remove(i) for i in sett.omit_list]
                    except:
                        pass
                    break
            
            p2.action(choice)
            
            # player 3
            choice = random.choice(choices)
            print("choice = {}".format(choice))
            choices.remove(choice)
            for sett in settlers.settlements:
                if sett.id == choice:

                    try:
                        [choices.remove(i) for i in sett.omit_list]
                    except:
                        pass
                    break
            
            p3.action(choice)
            
            # player 4
            choice = random.choice(choices)
            print("choice = {}".format(choice))
            choices.remove(choice)
            for sett in settlers.settlements:
                if sett.id == choice:

                    try:
                        [choices.remove(i) for i in sett.omit_list]
                    except:
                        pass
                    break
            
            p4.action(choice)

            done = True
            
#         elif event.type == MOUSEBUTTONDOWN:
#             click = pygame.mouse.get_pos()
#             x = (10*math.floor(click[0] / 10))-5
#             y = (10*math.floor(click[1] / 10))-5       
           
#             for sett in settlers.settlements:
#                 if sett.location == (x,y):
#                     choice = sett.id
#                     p1.action(choice)
#                     clicks -= 1

#                     for tile in sett.onTiles:
#                         print (str(resources[tile.resource]) + " is " + str(tile.value))

        
    # draw the board        
    settlers.draw()

    # draw the window onto the screen
    pygame.display.update()
    mainClock.tick(40)

pygame.quit()
